In [1]:
import requests
import json
import pandas as pd
import re
from pprint import pprint

# Import the API key
from config import apikey

In [12]:
# read oscar winning movie list
oscar_df = pd.DataFrame( pd.read_csv(".\Resources\oscar.csv", index_col='Unnamed: 0'))
oscar_df.dropna(inplace=True)
# limit to first 6 movies
oscar_df = oscar_df.loc[961:,:]

# rename oscar df to avoide overlap with imdb data
for name in oscar_df.columns :
    oscar_df = oscar_df.rename( columns={ name : f'Oscar_{name}' })
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination
961,Fantasia,1940,0,0
962,A Farewell to Arms,33,2,4
963,The Farmer's Daughter,1947,1,2
964,Father Goose,1964,1,3
965,Federico Fellini's 8½,1963,2,5
...,...,...,...,...
1355,The Yankee Doodle Mouse,1943,1,1
1356,The Yearling,1946,2,7
1357,"Yesterday, Today and Tomorrow",1964,1,1
1358,You Can't Take It with You,1938,2,7


In [13]:
oscar_df = oscar_df.reset_index(drop=True)

oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination
0,Fantasia,1940,0,0
1,A Farewell to Arms,33,2,4
2,The Farmer's Daughter,1947,1,2
3,Father Goose,1964,1,3
4,Federico Fellini's 8½,1963,2,5
...,...,...,...,...
393,The Yankee Doodle Mouse,1943,1,1
394,The Yearling,1946,2,7
395,"Yesterday, Today and Tomorrow",1964,1,1
396,You Can't Take It with You,1938,2,7


In [14]:
# omdb api preparation
base_url = "http://www.omdbapi.com/?"
params = { "apikey" : apikey, "type" : "movie" }

# general info to extract from api. column names are key in api.
general_col = ['Year', 'Rated', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', \
               'imdbRating', 'imdbVotes','BoxOffice']

for i in range(len(general_col)) :
    oscar_df[ general_col[i] ] = None

# non omdb rating to fulfill
ratings_col = ['Ratings_IMD', 'Ratings_Rotten_Tomatoes' ,'Ratings_Metacritic']

for i in range(len(ratings_col)) :
    oscar_df[ ratings_col[i] ] = None


In [15]:
# request API

for index in range(len(oscar_df)) :
    
    # export movie by film name in oscar df
    params["t"] = re.sub('\s+', '+', oscar_df.loc[ index , 'Oscar_Film' ] ) 
    movie_data = requests.get( base_url , params ).json()
    
    # extract general info and fulfill the dataframe, by colnames = keys
    for info_col in general_col :
        
        try :
            oscar_df.loc[ index , info_col ] = movie_data[ info_col ]
            
        except KeyError :
            oscar_df.loc[ index , info_col ] = None
            print( f'Film: {params["t"]}. Missing Key: {info_col}')
    
    # extract other ratings and fill rating columns
    try :
        temp_rating = pd.DataFrame( movie_data[ 'Ratings' ] ).set_index("Source")
    except KeyError :
        print( f'Film: {params["t"]}. Missing Other Ratings')
        oscar_df.loc[ index , 'Ratings_IMD'] = None
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None
        oscar_df.loc[ index , 'Ratings_Metacritic'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_IMD'] = temp_rating.loc[ "Internet Movie Database" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_IMD'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = temp_rating.loc[ "Rotten Tomatoes" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_Rotten_Tomatoes'] = None
        
    try :
        oscar_df.loc[ index , 'Ratings_Metacritic'] = temp_rating.loc["Metacritic" ,'Value']
    except KeyError :
        oscar_df.loc[ index , 'Ratings_Metacritic'] = None


Film: Helen+Keller+in+Her+Story. Missing Key: Year
Film: Helen+Keller+in+Her+Story. Missing Key: Rated
Film: Helen+Keller+in+Her+Story. Missing Key: Runtime
Film: Helen+Keller+in+Her+Story. Missing Key: Genre
Film: Helen+Keller+in+Her+Story. Missing Key: Director
Film: Helen+Keller+in+Her+Story. Missing Key: Writer
Film: Helen+Keller+in+Her+Story. Missing Key: Actors
Film: Helen+Keller+in+Her+Story. Missing Key: Plot
Film: Helen+Keller+in+Her+Story. Missing Key: Language
Film: Helen+Keller+in+Her+Story. Missing Key: Country
Film: Helen+Keller+in+Her+Story. Missing Key: Awards
Film: Helen+Keller+in+Her+Story. Missing Key: imdbRating
Film: Helen+Keller+in+Her+Story. Missing Key: imdbVotes
Film: Helen+Keller+in+Her+Story. Missing Key: BoxOffice
Film: Helen+Keller+in+Her+Story. Missing Other Ratings
Film: In+Beaver+Valley. Missing Key: Year
Film: In+Beaver+Valley. Missing Key: Rated
Film: In+Beaver+Valley. Missing Key: Runtime
Film: In+Beaver+Valley. Missing Key: Genre
Film: In+Beaver+Vall

In [7]:
movie_data

{'Title': 'Forbidden Games',
 'Year': '1952',
 'Rated': 'Not Rated',
 'Released': '08 Dec 1952',
 'Runtime': '86 min',
 'Genre': 'Comedy, Drama, War',
 'Director': 'René Clément',
 'Writer': 'François Boyer, Jean Aurenche, Pierre Bost',
 'Actors': 'Georges Poujouly, Brigitte Fossey, Amédée',
 'Plot': 'A young French girl orphaned in a Nazi air attack is befriended by the son of a poor farmer, and together they try to come to terms with the realities of death.',
 'Language': 'French',
 'Country': 'France',
 'Awards': 'Nominated for 1 Oscar. 8 wins & 1 nomination total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMzE4NTc0MDc1NV5BMl5BanBnXkFtZTgwODAxMTEyNTE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.0/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '100%'}],
 'Metascore': 'N/A',
 'imdbRating': '8.0',
 'imdbVotes': '12,699',
 'imdbID': 'tt0043686',
 'Type': 'movie',
 'DVD': '06 Dec 2005',
 'BoxOffice': '$19,889',
 'Production': 'N/A',
 'Website'

In [16]:
oscar_df

,Oscar_Film,Oscar_Year,Oscar_Award,Oscar_Nomination,Year,Rated,Runtime,Genre,Director,Writer,...,Plot,Language,Country,Awards,imdbRating,imdbVotes,BoxOffice,Ratings_IMD,Ratings_Rotten_Tomatoes,Ratings_Metacritic
0,Fantasia,1940,0,0,1940,G,124 min,"Animation, Family, Fantasy","James Algar, Samuel Armstrong, Ford Beebe Jr.","Joe Grant, Dick Huemer, Lee Blair",...,A collection of animated interpretations of gr...,English,United States,8 wins & 1 nomination,7.7,"99,747","$76,408,097",7.7/10,95%,96/100
1,A Farewell to Arms,33,2,4,1932,Unrated,80 min,"Drama, Romance, War",Frank Borzage,"Benjamin Glazer, Oliver H.P. Garrett, Ernest H...",...,An American ambulance driver and an English nu...,"English, Italian, Latin, French",United States,Won 2 Oscars. 4 wins & 2 nominations total,6.4,"6,472",N/A,6.4/10,94%,None
2,The Farmer's Daughter,1947,1,2,1947,Unrated,97 min,"Comedy, Drama, Romance",H.C. Potter,"Allen Rivkin, Laura Kerr, Hella Wuolijoki",...,After leaving her family's farm to study nursi...,"English, Swedish",United States,Won 1 Oscar. 1 win & 1 nomination total,7.2,"2,736",N/A,7.2/10,100%,None
3,Father Goose,1964,1,3,1964,Approved,118 min,"Adventure, Comedy, Romance",Ralph Nelson,"Peter Stone, Frank Tarloff, S.H. Barnett",...,"During World War II, a man persuaded to live o...","English, French, Japanese",United States,Won 1 Oscar. 2 wins & 5 nominations total,7.3,"11,469","$12,500,000",7.3/10,80%,None
4,Federico Fellini's 8½,1963,2,5,2001,N/A,8 min,"Documentary, Short",N/A,N/A,...,Filmmaker Terry Gilliam discusses Fellini's 8&...,English,USA,N/A,8.1,10,N/A,8.1/10,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,The Yankee Doodle Mouse,1943,1,1,1943,G,8 min,"Animation, Short, Comedy","Joseph Barbera, William Hanna","William Hanna, Joseph Barbera, Cal Howard",...,"In honor of the U.S. military during WWII, Tom...",English,United States,Won 1 Oscar. 1 win total,7.7,"1,917",N/A,7.7/10,None,None
394,The Yearling,1946,2,7,1946,Approved,128 min,"Drama, Family, Western",Clarence Brown,"Paul Osborn, Marjorie Kinnan Rawlings, John Le...",...,A boy persuades his parents to allow him to ad...,English,United States,Won 2 Oscars. 3 wins & 5 nominations total,7.2,"6,162",N/A,7.2/10,100%,89/100
395,"Yesterday, Today and Tomorrow",1964,1,1,1963,Not Rated,119 min,"Comedy, Romance",Vittorio De Sica,"Eduardo De Filippo, Isabella Quarantotti, Albe...",...,Stories about three very different women and t...,Italian,"Italy, France",Won 1 Oscar. 8 wins & 3 nominations total,7.2,"9,658",N/A,7.2/10,78%,None
396,You Can't Take It with You,1938,2,7,1938,Passed,126 min,"Comedy, Drama, Romance",Frank Capra,"Robert Riskin, George S. Kaufman, Moss Hart",...,The son of a snobbish Wall Street banker becom...,"English, Russian",United States,Won 2 Oscars. 2 wins & 7 nominations total,7.8,"26,756",N/A,7.8/10,95%,None


In [18]:
oscar_df.to_csv('.\Resources\oscar_omdb_2.csv', index=False)